## Imports

In [1]:
import googlemaps
import pandas as pd
import numpy as np
import re
import os
import geopandas as gpd
import folium
import tabula

## Read-in

In [2]:
# Gotta figure out how to get rid of the java error.
# SO: https://stackoverflow.com/questions/54817211/java-command-is-not-found-from-this-python-process-please-ensure-java-is-inst

dsf = tabula.io.read_pdf('condo.pdf', pages='all',lattice=True)

## Clean Data

In [3]:
df = dsf[0]
df.columns = df.iloc[0]
df = df.drop(df.index[0])
### This next line gets ride of the Nan ###
df = df.iloc[: , 1:]
df = df.replace(np.nan, 'Not Available', regex=True)
df = df.rename(columns={'Building / Address / City': 'building_address_city','Days on\rMarket':'days_on_market',\
                        'Price /\rSq. Ft.':'price_per_sqft'})
df['geo_address'] = df['building_address_city']
df['geo_address'] = df['geo_address'].map(lambda x: re.sub(r'\r', ' ', x))
df = df.replace('\n',' ',regex=True)
df2 = df['geo_address'].str.extract(r'(?P<building_name>\D*)(?P<address_only>\d.*)', expand=True)

Review this later (it's about how to properly merge DFs): https://pandas.pydata.org/pandas-docs/stable/user_guide/merging.html

## Geocode

In [4]:
result = pd.merge(df,df2,left_index=True, right_index=True)
df = result

In [5]:
%store -r google_maps_API_Key
gmaps_key = googlemaps.Client(key=google_maps_API_Key)

In [6]:
def geocode(add):
    g = gmaps_key.geocode(add)
    lat = g[0]["geometry"]["location"]["lat"]
    lng = g[0]["geometry"]["location"]["lng"]
    return (lat, lng)

df['geocoded'] = df['geo_address'].apply(geocode)

In [7]:
df['geocoded'] = df['geocoded'].astype(str)
df[['lat', 'lon']] = df['geocoded'].str.strip('()').str.split(', ', expand=True)
df['lat'] = df['lat'].astype(float)
df['lon'] = df['lon'].astype(float)

Folium documentation link: https://python-visualization.github.io/folium/index.html

Folium is the library that lets us use leaflet with Python, since it's meant to be used with JavaScript.

For adding titles: https://stackoverflow.com/questions/61928013/adding-a-title-or-text-to-a-folium-map

## Correction section

In [8]:
df

,building_address_city,Close Date,days_on_market,Sale Price,Sq. Ft.,price_per_sqft,Agent,Listing Broker,Buyer Agent,Buyer Broker,geo_address,building_name,address_only,geocoded,lat,lon
1,The Grove at Grand Bay\r2669 S Bayshore Dr 303...,4/27/2023,231,"$5,025,000.00",4100,"$1,225.61",Consuelo Stewart,Shelton and Stewart REALTORS,Tristan Alexander,"Compass Florida, LLC",The Grove at Grand Bay 2669 S Bayshore Dr 303N...,The Grove at Grand Bay,2669 S Bayshore Dr 303N Miami,"(25.7306755, -80.2371172)",25.730676,-80.237117
2,St Regis Bal Harbour\r9701 Collins Ave 704S\rB...,4/28/2023,3,"$4,450,000.00",2505,"$2,328.62",Scott Gerow,"Compass Florida, LLC",Jelena Grady,Trust Invest Real Estate Corp,St Regis Bal Harbour 9701 Collins Ave 704S Bal...,St Regis Bal Harbour,9701 Collins Ave 704S Bal Harbour,"(25.8884913, -80.1228812)",25.888491,-80.122881
3,Ocean Tower Two Condo\r791 Crandon Blvd 706\rK...,4/24/2023,187,"$4,200,000.00",3084,"$1,361.87",Monica Slodarz,"Compass Florida, LLC",Monica Slodarz,"Compass Florida, LLC",Ocean Tower Two Condo 791 Crandon Blvd 706 Key...,Ocean Tower Two Condo,791 Crandon Blvd 706 Key Biscayne,"(25.6885968, -80.1581219)",25.688597,-80.158122
4,Four Seasons Residences\r1425 Brickell Ave 65B...,4/25/2023,400,"$3,200,000.00",1964,"$1,629.33",Karim Daneri,"KD Prime Properties Int'l, LLC",Phillip Delgado PA,RE/MAX Advance Realty II,Four Seasons Residences 1425 Brickell Ave 65B ...,Four Seasons Residences,1425 Brickell Ave 65B Miami,"(25.759088, -80.19181669999999)",25.759088,-80.191817
5,Two Park Grove Condo\r2821 S Bayshore Dr 11C\r...,4/27/2023,298,"$3,200,000.00",1954,"$1,637.67",Ana Jerez,Cervera Real Estate Inc.,Evan Goldman,RE/MAX Advance Realty II,Two Park Grove Condo 2821 S Bayshore Dr 11C Co...,Two Park Grove Condo,2821 S Bayshore Dr 11C Coconut Grove,"(25.7287205, -80.23808009999999)",25.728721,-80.238080
6,Ocean Four\r17201 Collins Ave 901\rSunny Isles...,4/27/2023,13,"$3,000,000.00",2755,"$1,088.93",Magda Ruby,MC Miami Realty LLC,Bruce Germinsky,"BG Realty Services of Florida, LLC",Ocean Four 17201 Collins Ave 901 Sunny Isles B...,Ocean Four,17201 Collins Ave 901 Sunny Isles Beach,"(25.9356109, -80.121168)",25.935611,-80.121168
7,Ocean Club Community\r791 Crandon Blvd 402\rKe...,4/24/2023,117,"$2,100,000.00",2510,$836.65,Monica Slodarz,"Compass Florida, LLC",Monica Slodarz,"Compass Florida, LLC",Ocean Club Community 791 Crandon Blvd 402 Key ...,Ocean Club Community,791 Crandon Blvd 402 Key Biscayne,"(25.688277, -80.158928)",25.688277,-80.158928
8,Nine Island Avenue\r9 Island Ave 2415\rMiami B...,4/26/2023,14,"$2,050,000.00",1837,"$1,115.95",William Hahne,Coldwell Banker Realty,Stefania Cambarau,Douglas Elliman,Nine Island Avenue 9 Island Ave 2415 Miami Beach,Nine Island Avenue,9 Island Ave 2415 Miami Beach,"(25.790337, -80.14877249999999)",25.790337,-80.148772
9,The Tiffany at Bal Harbour\r10175 Collins Ave ...,4/27/2023,158,"$1,975,000.00",2353,$839.35,Teodoro Palmieri,Elite Ocean View Realty LLC,Teodoro Palmieri,Elite Ocean View Realty LLC,The Tiffany at Bal Harbour 10175 Collins Ave 7...,The Tiffany at Bal Harbour,10175 Collins Ave 701 Bal Harbour,"(25.8937855, -80.1231004)",25.893785,-80.123100
10,0Williams Island\r4000 Island Blvd 2506\rAventura,4/28/2023,332,"$1,900,000.00",3335,$569.72,Lazaro Rivero,"Rivero Real Estate , Inc.",Margo Johnson,Beachfront Realty Inc,0Williams Island 4000 Island Blvd 2506 Aventura,,0Williams Island 4000 Island Blvd 2506 Aventura,"(25.9416063, -80.1348359)",25.941606,-80.134836


In [9]:
df.at[10,'building_name']=('Williams Island')
df.at[10,'address_only']=('4000 Island Boulevard')

## Format Data

In [10]:
### Insert NaNs if needed ###
df = df.replace('N/A', np.nan)

In [11]:
df['int_Sale_Price'] = df['Sale Price'].str.replace('$','',regex=True)
df['int_Sale_Price'] = df['int_Sale_Price'].str.replace(',','',regex=True)
df['int_Sale_Price'] = pd.to_numeric(df['int_Sale_Price'])

df['price_per_sqft'] = df['price_per_sqft'].str.replace('$','',regex=True)
df['price_per_sqft'] = df['price_per_sqft'].str.replace(',','',regex=True)
df['price_per_sqft'] = pd.to_numeric(df['price_per_sqft'])

df['days_on_market'] = pd.to_numeric(df['days_on_market'])

## Color-code top sale

In [12]:
### Insert RANK values ###
df['RANK'] = range(1, len(df) + 1)
# use numpy to assign values to the 'COLOR' column
df['COLOR'] = np.where(df['RANK'] <= 1, 'orange', 'blue')

## HTML Popup Formatter

In [13]:
df.columns

Index(['building_address_city', 'Close Date', 'days_on_market', 'Sale Price',
       'Sq. Ft.', 'price_per_sqft', 'Agent', 'Listing Broker', 'Buyer Agent',
       'Buyer Broker', 'geo_address', 'building_name', 'address_only',
       'geocoded', 'lat', 'lon', 'int_Sale_Price', 'RANK', 'COLOR'],
      dtype='object')

In [14]:
def popup_html(row):
    building_name = row['building_name']
    price = row['Sale Price']
    days_on_market = row['days_on_market']
    listing_agent = row['Agent']
    buyers_agent = row['Buyer Agent']
    psf = row['price_per_sqft']
    address = row['address_only']
    
    html = '''<!DOCTYPE html>
    <html>
    <strong>Building Name: </strong>{}'''.format(building_name) + '''<br>
    <strong>Sale Price: </strong>{}'''.format(price) + '''<br>
    <strong>Days on Market: </strong>{}'''.format(days_on_market) + '''<br>
    <strong>Listing Agent: </strong>{}'''.format(listing_agent) + '''<br>
    <strong>Buyer's Agent: </strong>{}'''.format(buyers_agent) + '''<br>
    <strong>Price sq ft: </strong>${}'''.format(psf) + '''<br>
    <strong>Address: </strong>{}'''.format(address) + '''<br>
    </html>
    '''
    return html

In [15]:
import folium
from folium.plugins import MarkerCluster

m = folium.Map(location=df[["lat", "lon"]].mean().to_list(), zoom_start=10)

title_html = '''
              <h3 align="center" style="font-size:16px"><b>{}</b></h3>
             '''.format(f'Recent Miami-Dade Condo Sales ')

caption_html = '''
                <p align="center" style="vertical-align: bottom; font-size:13px"><i>{}</i></p>
                '''.format('April 23rd - April 29th')


### Create map container ###
m = folium.Map(location=df[["lat", "lon"]].mean().to_list(),zoom_start=9.5,tiles=None)

# Create two FeatureGroups for different color pins
fg_blue = folium.FeatureGroup(name='All other sales')
fg_orange = folium.FeatureGroup(name='Top Sale')

for index, row in df.iterrows():
    # Add the markers to the appropriate FeatureGroup based on the color
    if row['COLOR'] == 'blue':
        marker = folium.Marker(
            location=[row['lat'], row['lon']],
            radius=5,
            fill=True,
            icon=folium.Icon(color=row['COLOR']),
            popup=folium.Popup(popup_html(row), max_width=400))
        marker.add_to(fg_blue)
    else:
        marker = folium.Marker(
            location=[row['lat'], row['lon']],
            radius=5,
            fill=True,
            icon=folium.Icon(color=row['COLOR']),
            popup=folium.Popup(popup_html(row), max_width=400))
        marker.add_to(fg_orange)

# Add the FeatureGroups to the map
fg_orange.add_to(m)
fg_blue.add_to(m)

folium.TileLayer('OpenStreetMap',control=False).add_to(m)

# Add LayerControl to the map
folium.map.LayerControl(collapsed=False).add_to(m)
m.get_root().html.add_child(folium.Element(title_html))
m.get_root().html.add_child(folium.Element(caption_html))
            
# Display map
m

In [16]:
m.save('index.html')

## Data snagger

In [17]:
### Set up formatting ###
BR = '\n'

ME = '\033[1m' + 'Most Expensive' + '\033[0m'
LE = '\033[1m' + 'Least Expensive' + '\033[0m'

MAX_PSF = '\033[1m' + 'Highest Price Per Square Foot' + '\033[0m'
MIN_PSF = '\033[1m' + 'Lowest Price Per Square Foot' + '\033[0m'

DAYS_MAX = '\033[1m' + 'Most Days on Market' + '\033[0m'
DAYS_MIN = '\033[1m' + 'Fewest Days on Market' + '\033[0m'

In [18]:
### Highest and lowest sale price ###
print(f"{ME}{BR}{df.loc[df['int_Sale_Price'].idxmax()]['building_name']}, {df.loc[df['int_Sale_Price'].idxmax()]['address_only']} | Price ${df.loc[df['int_Sale_Price'].idxmax()]['int_Sale_Price']:,.0f} | ${df.loc[df['int_Sale_Price'].idxmax()]['price_per_sqft']:,.0f} psf | Listing agent: {df.loc[df['int_Sale_Price'].idxmax()]['Agent']} with {df.loc[df['int_Sale_Price'].idxmax()]['Listing Broker']} | Buyer's agent: {df.loc[df['int_Sale_Price'].idxmax()]['Buyer Agent']} with {df.loc[df['int_Sale_Price'].idxmax()]['Buyer Broker']} | Days on market: {df.loc[df['int_Sale_Price'].idxmax()]['days_on_market']}")
print(f"{LE}{BR}{df.loc[df['int_Sale_Price'].idxmin()]['building_name']}, {df.loc[df['int_Sale_Price'].idxmin()]['address_only']} | Price ${df.loc[df['int_Sale_Price'].idxmin()]['int_Sale_Price']:,.0f} | ${df.loc[df['int_Sale_Price'].idxmin()]['price_per_sqft']:,.0f} psf | Listing agent: {df.loc[df['int_Sale_Price'].idxmin()]['Agent']} with {df.loc[df['int_Sale_Price'].idxmin()]['Listing Broker']} | Buyer's agent: {df.loc[df['int_Sale_Price'].idxmin()]['Buyer Agent']} with {df.loc[df['int_Sale_Price'].idxmin()]['Buyer Broker']} | Days on market: {df.loc[df['int_Sale_Price'].idxmin()]['days_on_market']}")
### Highest and lowest psf ###
print(f"{MAX_PSF}{BR}{df.loc[df['price_per_sqft'].idxmax()]['building_name']}, {df.loc[df['price_per_sqft'].idxmax()]['address_only']} | Price ${df.loc[df['price_per_sqft'].idxmax()]['int_Sale_Price']:,.0f} | ${df.loc[df['price_per_sqft'].idxmax()]['price_per_sqft']:,.0f} psf | Listing agent: {df.loc[df['price_per_sqft'].idxmax()]['Agent']} with {df.loc[df['price_per_sqft'].idxmax()]['Listing Broker']} | Buyer's agent: {df.loc[df['price_per_sqft'].idxmax()]['Buyer Agent']} with {df.loc[df['price_per_sqft'].idxmax()]['Buyer Broker']} | Days on market: {df.loc[df['price_per_sqft'].idxmax()]['days_on_market']}")
print(f"{MIN_PSF}{BR}{df.loc[df['price_per_sqft'].idxmin()]['building_name']}, {df.loc[df['price_per_sqft'].idxmin()]['address_only']} | Price ${df.loc[df['price_per_sqft'].idxmin()]['int_Sale_Price']:,.0f} | ${df.loc[df['price_per_sqft'].idxmin()]['price_per_sqft']:,.0f} psf | Listing agent: {df.loc[df['price_per_sqft'].idxmin()]['Agent']} with {df.loc[df['price_per_sqft'].idxmin()]['Listing Broker']} | Buyer's agent: {df.loc[df['price_per_sqft'].idxmin()]['Buyer Agent']} with {df.loc[df['price_per_sqft'].idxmin()]['Buyer Broker']} | Days on market: {df.loc[df['price_per_sqft'].idxmin()]['days_on_market']}")
### Highest and lowest days on market ###
print(f"{DAYS_MAX}{BR}{df.loc[df['days_on_market'].idxmax()]['building_name']}, {df.loc[df['days_on_market'].idxmax()]['address_only']} | Price ${df.loc[df['days_on_market'].idxmax()]['int_Sale_Price']:,.0f} | ${df.loc[df['days_on_market'].idxmax()]['price_per_sqft']:,.0f} psf | Listing agent: {df.loc[df['days_on_market'].idxmax()]['Agent']} with {df.loc[df['days_on_market'].idxmax()]['Listing Broker']} | Buyer's agent: {df.loc[df['days_on_market'].idxmax()]['Buyer Agent']} with {df.loc[df['days_on_market'].idxmax()]['Buyer Broker']} | Days on market: {df.loc[df['days_on_market'].idxmax()]['days_on_market']}")
print(f"{DAYS_MIN}{BR}{df.loc[df['days_on_market'].idxmin()]['building_name']}, {df.loc[df['days_on_market'].idxmin()]['address_only']} | Price ${df.loc[df['days_on_market'].idxmin()]['int_Sale_Price']:,.0f} | ${df.loc[df['days_on_market'].idxmin()]['price_per_sqft']:,.0f} psf | Listing agent: {df.loc[df['days_on_market'].idxmin()]['Agent']} with {df.loc[df['days_on_market'].idxmin()]['Listing Broker']} | Buyer's agent: {df.loc[df['days_on_market'].idxmin()]['Buyer Agent']} with {df.loc[df['days_on_market'].idxmin()]['Buyer Broker']} | Days on market: {df.loc[df['days_on_market'].idxmin()]['days_on_market']}")

Most Expensive
The Grove at Grand Bay , 2669 S Bayshore Dr 303N Miami | Price $5,025,000 | $1,226 psf | Listing agent: Consuelo Stewart with Shelton and Stewart REALTORS | Buyer's agent: Tristan Alexander with Compass Florida, LLC | Days on market: 231
Least Expensive
Williams Island, 4000 Island Boulevard | Price $1,900,000 | $570 psf | Listing agent: Lazaro Rivero with Rivero Real Estate , Inc. | Buyer's agent: Margo Johnson with Beachfront Realty Inc | Days on market: 332
Highest Price Per Square Foot
St Regis Bal Harbour , 9701 Collins Ave 704S Bal Harbour | Price $4,450,000 | $2,329 psf | Listing agent: Scott Gerow with Compass Florida, LLC | Buyer's agent: Jelena Grady with Trust Invest Real Estate Corp | Days on market: 3
Lowest Price Per Square Foot
Williams Island, 4000 Island Boulevard | Price $1,900,000 | $570 psf | Listing agent: Lazaro Rivero with Rivero Real Estate , Inc. | Buyer's agent: Margo Johnson with Beachfront Realty Inc | Days on market: 332
Most Days on Market
F

## Map URL snagger

Map template URL: `https://trd-digital.github.io/trd-news-interactive-maps/{map-folder-name}`

In [19]:
base_name = 'https://trd-digital.github.io/trd-news-interactive-maps/'

In [20]:
cwd = os.getcwd()

cwd = cwd.split('/')

final_name = base_name + cwd[-1]
print(final_name)

https://trd-digital.github.io/trd-news-interactive-maps/condo_sales_week_ending_04222023
